# 패키지 선언

In [5]:
# 패키지 불러오기

import requests
import bs4
from bs4 import BeautifulSoup
import os
import random
import hashlib
import urllib.request
import urllib
from urllib.parse import urlparse, urlencode, parse_qs
import pandas as pd
import csv
import re

from datetime import datetime, timedelta
from urllib import parse
from openpyxl import load_workbook
import time
import requests
import numpy as np
import json
from random import *
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent

from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException

ua = UserAgent()
userAgent = ua.random
print(userAgent)

options = Options()
options.add_argument(f"user-agent={userAgent}")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')

executable_path = '/home/ubuntu/book_crawling/geckodriver' # chrome driver의 경로
driver = webdriver.Firefox(executable_path=executable_path, options=options)

Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


# 영풍문고 크롤링 함수

In [6]:
def parse_YP(isbn):
    
    book = {'isbn' : isbn,
            '제목' : '없음',
            '저자' : '없음',
            '역자' : '없음',
            '별점' : '없음',
            '리뷰' : '없음',
            '카테고리' : '없음',
            '책소개' : '없음',
            '저자소개' : '없음',
            '추천사' : '없음',
            '목차' : '없음',
            '책속으로' : '없음',
            '출판사리뷰' : '없음'
           }
    
    search_page = 'https://www.ypbooks.co.kr/search.yp?query='
    driver.get(search_page + isbn)
    link = driver.find_element_by_class_name('recom').find_element_by_tag_name('a')
    link.click()
    
    driver.implicitly_wait(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

     # 제목
    try:
        title_el = soup.select_one('.bookNameTitle')
        title = title_el.text.strip()
    except:
        title = '없음'
    book['제목'] = title

    # 저자, 역자
    try:
        info_bar = soup.select_one('.mT10.prodInfo-dd')
        author = re.search('저자[^출판사]*', info_bar.text).group().replace('ㅣ', '').strip()
        try:
            translator = re.search('역자[^2|1]*', info_bar.text).group().replace('ㅣ', '').strip()
        except:
            translator = '없음'
    except:
        author = '없음'
    book['저자'] = author
    book['역자'] = translator
    
    #  info_tabs
    info_tabs = soup.select('.infoTab')
    contents = info_tabs[0].find_next_siblings()
    contents_f = list(filter(lambda x: 'tabCon' in str(x), contents))
    
    # 별점, 리뷰
    book['별점'] = '없음'
    book['리뷰'] = '없음'
    book['추천사'] = '없음'
      
    category = '없음'
    book_info = '없음'
    in_book = '없음'
    table = '없음'
    pub_review = '없음'
    author_info = '없음'
    translator_info = '없ㅇ므'
    
    for i in range(len(contents_f)):
        # 카테고리
        if '분류' in contents_f[i].find_previous_siblings('ul')[0].select_one('.on').text:
            category_el = contents_f[i]
            category = category_el.text.strip()
        
        # 책소개
        if '도서정보' in contents_f[i].find_previous_siblings('ul')[0].select_one('.on').text:
            book_info_el = contents_f[i]
            book_info = book_info_el.text.replace('펼쳐보기', '').strip()
        
        # 목차, 책속으로
        if '목차' in contents_f[i].find_previous_siblings('ul')[0].select_one('.on').text:
            table_el = contents_f[i]
            table_t = table_el.text.replace('펼쳐보기', '').replace('이미지 출처', '').replace('\t', '').strip()
            in_book = '없음'
            if '본 문' in table_t:
                in_book = re.search('\[본\s?문\](.|\s)*', table_t).group().replace('[본 문]\n\n', '')
                table = table_t[:table_t.find('[본 문]')].replace('[목 차]\n\n', '').strip()
            else:
                table = table_t
        
        # 출판사리뷰
        if '출판사서평' in contents_f[i].find_previous_siblings('ul')[0].select_one('.on').text:
            pub_review_el = contents_f[i]
            pub_review = pub_review_el.text.replace('펼쳐보기', '').strip()

        # 저자소개, 역자소개
        if '저자' in contents_f[i].find_previous_siblings('ul')[0].select_one('.on').text:
            author_info_el = contents_f[i]
            author_text = author_info_el.text.replace('펼쳐보기', '').strip()
            if '옮긴이' in author_text:
                translator_info = re.search('옮긴이(.|\s)*', author_text).group()
                author_info = author_text[:author_text.find('옮긴이')].strip()
            else:
                author_info = author_text
    
    book = {'isbn' : isbn,
            '제목' : title,
            '저자' : author,
            '역자' : translator,
            '별점' : '없음',
            '리뷰' : '없음',
            '카테고리' : category,
            '책소개' : book_info,
            '저자소개' : author_info,
            '추천사' : '없음',
            '목차' : table,
            '책속으로' : in_book,
            '출판사리뷰' : pub_review
            }
    
    return book

In [7]:
sample_isbn = '9791190779272'
parse_YP(sample_isbn)

{'isbn': '9791190779272',
 '제목': '진화의 오리진',
 '저자': '저자 : 존그리빈,메리그리빈',
 '역자': '역자 : 권루시안',
 '별점': '없음',
 '리뷰': '없음',
 '카테고리': '국내도서  > 자연 > 과학일반 > 자연교양물',
 '책소개': '『네이처』 선정 ‘최고의 과학저술가’ 존 그리빈의 신작,\n아리스토텔레스부터 DNA까지 다윈의 ‘위험한 생각’을 추적하다!\n『진화의 오리진』은 ‘진화’에 관한 흥미진진한 이야기다. 진화에 관한 과학적 사실을 발견해 나가는 과정을 명쾌한 구성과 통찰력을 갖춘 글로 풀어냈다. 존 그리빈과 공저자인 메리 그리빈은 ‘다윈의 위험한 생각’을 바탕으로 ‘진화’라는 방대한 주제에 대해 유전학, 지질학 등 각 분야의 세밀한 내용까지 넓고도 깊게 망라하여 설명하고 있다.\n☞ 선정 및 수상내역\n★★★ Waterstone 선정 2020년 최고의 책 ★★★',
 '저자소개': '존 그리빈\n존 그리빈은 케임브리지 대학교에서 천문학 박사 학위를 받고 『네이처』에서, 이어 『뉴 사이언티스트』지에서 저널리스트로 활동했다. 얼핏 어렵다는 인상을 주기 쉬운 과학 분야에 관한 이야기를 쉽고 재미있게 풀어내는 솜씨가 뛰어나, 영국 BBC 뉴스에서 그의 책 『슈뢰딩거의 고양이를 찾아서』를 가리켜 수학에 대한 관심을 되살리는 방법을 잘 보여 주는 사례라고 말한 일도 있다(2002). 과학자라기보다 소설처럼 읽을 수 있는 과학 도서 작가이자 과학을 바탕으로 하는 소설 작가라고 자신을 소개하는 그는 『다중우주를 찾아서』와 『우주』 등 수많은 베스트셀러를 썼고, 여러 나라에서 수많은 상을 받았다.\n\n메리 그리빈\n영국에서 활동하는 교육자이자 아동청소년 과학 도서 작가로서, 심리학을 전공한 뒤 특별한 보살핌이 필요한 4~16세 아동청소년들을 위해 20년 동안 일한 경력이 있다. 『쉬』, 『코스모폴리탄』, 『가디언』 등 여러 신문 잡지에 기고했고, 『시간과 우주』라는 책으로 을 받았을 정도로 어려운 개념을 쉽게

# csv파일 불러오기 & 크롤링

In [8]:
book_specs = pd.read_csv('./book_spec_korean_isbns.csv', low_memory=False)
isbns = book_specs.loc[:, '0'].tolist()

len(isbns)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2872189

# 대량 크롤링

In [ ]:
num = 270000
count_save = num - 10000
results = []
for isbn in tqdm(isbns[num-10000:num]):
    time.sleep(2)
    try:
        result = parse_YP(isbn)
        results.append(result)
    except:
        result = {'isbn' : isbn,
                  '제목' : '없음',
                  '저자' : '없음',
                  '역자' : '없음',
                  '별점' : '없음',
                  '리뷰' : '없음',
                  '카테고리' : '없음',
                  '책소개' : '없음',
                  '저자소개' : '없음',
                  '추천사' : '없음',
                  '목차' : '없음',
                  '책속으로' : '없음',
                  '출판사리뷰' : '없음'
                 }
        results.append(result)

    count_save += 1
    
    if count_save % 1000 == 0:
        file_name = f'YP_{count_save}_287,000.csv'
        results_df = pd.DataFrame(results)
        results_df.to_csv('./crawled_csv/' + file_name, encoding='utf-8-sig')
        results = []
        print(f'saving {count_save}th results to {file_name}')
        
print('Crawling Completed')

saving 261000th results to YP_261000_287,000.csv
saving 262000th results to YP_262000_287,000.csv
saving 263000th results to YP_263000_287,000.csv
saving 264000th results to YP_264000_287,000.csv
saving 265000th results to YP_265000_287,000.csv
saving 266000th results to YP_266000_287,000.csv
